## Example: Particle Mesh Interpolation

### What Features Are We Using

* Mesh data
* Particle data
* Domain Decomposition

Thus far we have only encountered mesh data. This example shows how to use particle data 
and how to interpolate between the particles and the mesh.

To tell amrex how many particle components we want, pass in template parameters to `amrex::ParticleContainer`.
    
```C++
  typedef ParticleContainer<1 + 2*AMREX_SPACEDIM> MyParticleContainer;
  MyParticleContainer myPC(geom, dmap, ba);
```

Here we have asked for `1 + 2*AMREX_SPACEDIM` real components per particle, stored in Array-of-Struct fashion.
These will represent the mass, three velocity components, and three acceleration components of each particle.

We initialize the particle positions randomly throughout the domain, based on a target average number per cell.

Particle-mesh operations can be done with the `amrex::ParticleToMesh` and `amrex::MeshToParticle` routines. The type of interpolation to perform is passed in as a lambda function:

```C++
  amrex::ParticleToMesh(myPC, partMF, 0,
      [=] AMREX_GPU_DEVICE (const MyParticleContainer::ParticleType& p,
                            amrex::Array4<amrex::Real> const& rho)
      {
          amrex::Real lx = (p.pos(0) - plo[0]) * dxi[0] + 0.5;
          amrex::Real ly = (p.pos(1) - plo[1]) * dxi[1] + 0.5;
          amrex::Real lz = (p.pos(2) - plo[2]) * dxi[2] + 0.5;

          int i = amrex::Math::floor(lx);
          int j = amrex::Math::floor(ly);
          int k = amrex::Math::floor(lz);

          amrex::Real xint = lx - i;
          amrex::Real yint = ly - j;
          amrex::Real zint = lz - k;

          amrex::Real sx[] = {1.-xint, xint};
          amrex::Real sy[] = {1.-yint, yint};
          amrex::Real sz[] = {1.-zint, zint};

          for (int kk = 0; kk <= 1; ++kk) {
              for (int jj = 0; jj <= 1; ++jj) {
                  for (int ii = 0; ii <= 1; ++ii) {
                      amrex::Gpu::Atomic::AddNoRet(&rho(i+ii-1, j+jj-1, k+kk-1, 0),
                                              sx[ii]*sy[jj]*sz[kk]*p.rdata(0));
                  }
              }
          }
      }
```

The above code is for "Cloud in cell" interpolation. Under the hood, the deposition algorithm will be changed based on the whether the parallel backend is targetting GPUs or CPUs.

## Running the code

The simulation can be ran as `./04_ParticleMesh inputs`. 

The following inputs parameters could be tweaked:

```
nx            = 128 # number of grid points along the x axis
ny            = 128 # number of grid points along the y axis
nz            = 128 # number of grid points along the z axis
max_grid_size = 32  # grid size used for domain decomposition
nppc          = 10  # average number of particles per cell     

```

In [ ]:
import yt

In [ ]:
ds = yt.load("plot")

In [ ]:
sl = yt.SlicePlot(ds, 2, 'density')

In [ ]:
sl.annotate_grids()

In [ ]:
ad = ds.all_data()
ad.quantities.weighted_average_quantity('density', 'cell_volume')